In [1]:
import torch
import numpy
import json
from tqdm import tqdm
import os
import gc
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
from vllm import LLM, SamplingParams, LLMEngine

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-26 18:14:07,681	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
num_gpus = torch.cuda.device_count()
print(num_gpus)

2


In [5]:
from datasets import load_dataset
task_name='CodeSearchNet-python'
# Load the dataset
queries_corpus_dataset = load_dataset(f"CoIR-Retrieval/{task_name}-queries-corpus")
qrels_dataset = load_dataset(f"CoIR-Retrieval/{task_name}-qrels")

Generating test split: 100%|██████████| 14918/14918 [00:00<00:00, 899119.53 examples/s]


In [5]:
pd.set_option("display.max_colwidth",500)

In [23]:
pd.DataFrame(queries_corpus_dataset['corpus'])

,_id,title,partition,text,language,meta_information
0,c0,,train,"Set the text for this element.\n\n Arguments:\n text (str): The text\n cls (str): The class of the text, defaults to ``current`` (leave this unless you know what you are doing). There may be only one text content element of each class associated with the element.",,{'resource': ''}
1,c1,,train,Associate a document with this element.\n\n Arguments:\n doc (:class:`Document`): A document\n\n Each element must be associated with a FoLiA document.,,{'resource': ''}
2,c2,,train,"Tests whether a new element of this class can be added to the parent.\n\n This method is mostly for internal use.\n This will use the ``OCCURRENCES`` property, but may be overidden by subclasses for more customised behaviour.\n\n Parameters:\n parent (:class:`AbstractElement`): The element that is being added to\n set (str or None): The set\n raiseexceptions (bool): Raise an exception if the element can't be added?\n\n Returns:\n ...",,{'resource': ''}
3,c3,,train,This method will be called after an element is added to another and does some checks.\n\n It can do extra checks and if necessary raise exceptions to prevent addition. By default makes sure the right document is associated.\n\n This method is mostly for internal use.,,{'resource': ''}
4,c4,,train,Recompute textual value based on the text content of the children. Only supported on elements that are a ``TEXTCONTAINER``,,{'resource': ''}
...,...,...,...,...,...,...
280305,c280305,,test,Return a canvas from a grayscale image.\n\n :param im: single channel image\n :channels_on: channels to populate with input image\n :returns: :class:`jicbioimage.illustrate.Canvas`,,{'resource': ''}
280306,c280306,,test,Returns a unique ID of a given length.\n User `version=2` for cross-systems uniqueness.,,{'resource': ''}
280307,c280307,,test,Build a unique key from get data,,{'resource': ''}
280308,c280308,,test,Returns domain name portion of a URL,,{'resource': ''}


In [33]:
queries = pd.DataFrame(queries_corpus_dataset['queries'])
corpus = pd.DataFrame(queries_corpus_dataset['corpus'])

In [43]:
queries.tail()

,_id,title,partition,text,language,meta_information
280647,q280647,AnnotatedImage.from_grayscale,test,"def from_grayscale(im, channels_on=(True, True, True)):\n """"""Return a canvas from a grayscale image.\n\n :param im: single channel image\n :channels_on: channels to populate with input image\n :returns: :class:`jicbioimage.illustrate.Canvas`\n """"""\n xdim, ydim = im.shape\n canvas = np.zeros((xdim, ydim, 3), dtype=np.uint8)\n for i, include in enumerate(channels_on):\n if include:\n canvas[:, :, i] = im\n ...",python,{'resource': ''}
280648,q280648,get_uuid,test,"def get_uuid(length=32, version=1):\n """"""\n Returns a unique ID of a given length.\n User `version=2` for cross-systems uniqueness.\n """"""\n if version == 1:\n return uuid.uuid1().hex[:length]\n else:\n return uuid.uuid4().hex[:length]",python,{'resource': ''}
280649,q280649,get_unique_key_from_get,test,"def get_unique_key_from_get(get_dict):\n """"""\n Build a unique key from get data\n """"""\n site = Site.objects.get_current()\n key = get_dict_to_encoded_url(get_dict)\n cache_key = '{}_{}'.format(site.domain, key)\n return hashlib.md5(cache_key).hexdigest()",python,{'resource': ''}
280650,q280650,get_domain,test,"def get_domain(url):\n """""" Returns domain name portion of a URL """"""\n if 'http' not in url.lower():\n url = 'http://{}'.format(url)\n return urllib.parse.urlparse(url).hostname",python,{'resource': ''}
280651,q280651,get_url_args,test,"def get_url_args(url):\n """""" Returns a dictionary from a URL params """"""\n url_data = urllib.parse.urlparse(url)\n arg_dict = urllib.parse.parse_qs(url_data.query)\n return arg_dict",python,{'resource': ''}


In [35]:
corpus.shape, queries.shape

((280310, 6), (280652, 6))

In [48]:
qrels = pd.DataFrame(qrels_dataset['valid'])

In [49]:
qrels.head()

,query_id,corpus_id,score
0,q251820,c251820,1
1,q251821,c251821,1
2,q251822,c251822,1
3,q251823,c251823,1
4,q251824,c251824,1


In [ ]:
qrels['score'].unique()

array([1])

In [41]:
qrels[qrels['corpus_id']=='c280308']

,query_id,corpus_id,score
14916,q280650,c280308,1


In [66]:
def dataset_generation(qrels, queries, corpus):
    temp1 = queries[['_id', 'text']].rename(columns={'_id': 'query_id', 'text': 'code'})
    temp2 = corpus[['_id', 'text']].rename(columns={'_id': 'corpus_id', 'text': 'doc'})
    df_corpus = pd.merge(qrels, temp2, on='corpus_id', how='inner')
    df_codesearchdata = pd.merge(df_corpus, temp1, on='query_id', how='inner')
    
    return df_codesearchdata


In [67]:
queries = pd.DataFrame(queries_corpus_dataset['queries'])
corpus = pd.DataFrame(queries_corpus_dataset['corpus'])


In [68]:
queries_filter = queries[queries['partition']=='train']
corpus_filter = corpus[corpus['partition']=='train']
qrels_train = pd.DataFrame(qrels_dataset['train'])

train_df = dataset_generation(qrels_train,queries_filter, corpus_filter)

In [69]:
queries_filter = queries[queries['partition']=='test']
corpus_filter = corpus[corpus['partition']=='test']
qrels_test = pd.DataFrame(qrels_dataset['test'])

test_df = dataset_generation(qrels_test,queries_filter, corpus_filter)

In [75]:
queries_filter = queries[queries['partition']=='validation']
corpus_filter = corpus[corpus['partition']=='validation']
qrels_valid = pd.DataFrame(qrels_dataset['valid'])

valid_df = dataset_generation(qrels_valid,queries_filter, corpus_filter)

In [71]:
qrels_train.shape, train_df.shape

((251820, 3), (251820, 5))

In [72]:
qrels_test.shape, test_df.shape

((14918, 3), (14702, 5))

In [76]:
qrels_valid.shape, valid_df.shape

((13914, 3), (13788, 5))

In [86]:
# Set environment variable to help with memory fragmentation.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


def clean_output(text,keyword):
    # keyword = "Answer:"
    index = text.rfind(keyword)  # Find the last occurrence of "Answer:"
    if index != -1:
        return text[index + len(keyword):].strip()  
    return text


class ExplanationGeneratorLama:
    def __init__(self, model_name, max_new_tokens=500):
        # self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # self.tokenizer.padding_side = "left"
        
        self.max_new_tokens = max_new_tokens
        self.sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=max_new_tokens
        )
        self.llm = LLM(
            model=model_name,
            dtype="half"  # using half precision for GPU
        )
        # self.engine = LLMEngine(model_name, sampling_params=self.sampling_params)

    def generate_explanations_batch(self, entries, max_new_tokens=500):
        # Create prompts by combining each entry with each prompt template.
        
        prompts = []
        
        for entry in entries:
            prompt_templates = [
                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Provide a concise explanation of what the above doc and code mean. "
                "Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                , 

                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. "
                "Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                ,

                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. "
                "Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                ,

                f"Doc string: {entry['doc']}\n"
                f"Code snippet: {entry['code']}\n"
                "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. "
                "Please give the output just as text only. Do not return anything else.\n"
                "Answer: \n"
                ,

                f"Doc string - entry['doc'] : {entry['doc']}\n" \
                f"Code snippet - entry['code'] : {entry['code']}\n" \
                "Instruction: Explain how the code snippet in entry['code'] implements or achieves the functionality described in the doc string in entry['doc']. Please provide the explanation as text only without any additional content.\n" \
                "Answer: \n"
            ]
            
            for template in prompt_templates:
                prompt = (
                    f"Doc string: {entry['doc']}\n"
                    f"Code snippet: {entry['code']}\n"
                    f"{template}\n"
                    "Answer: \n"
                )
                prompts.append(prompt)
                
        # results = self.engine.generate(prompts)
        results = self.llm.generate(prompts, self.sampling_params)
        explanations = [result.outputs[0].text for result in results]
        
        # Regroup explanations by entry.
        n_prompts = len(prompt_templates)
        grouped_explanations = []
        for i in range(0, len(explanations), n_prompts):
            grouped_explanations.append(explanations[i:i+n_prompts])
            
        return grouped_explanations


In [87]:
import pandas as pd
import torch, gc
from tqdm import tqdm
# Import or define your ExplanationGeneratorLama class here

if __name__ == "__main__":
    csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/cosqa_queries_code_corpus.csv"  # change this to your CSV input path
    output_csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CoSQA_granite_explanations_vllm.csv"
    
    # Load the CSV file into a DataFrame
    # df = pd.read_csv(csv_path).head(20)
    df = test_df.head(10)
    # print("Data loaded from CSV")
    

    models_dict = {
        # "deepseek": "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
        'granite': "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"

    }    
    
    batch_size = 4
    
    for model_key, model_path in tqdm(models_dict.items(), desc="Processing models"):
        print(f"\nProcessing model {model_key}...")
        generator = ExplanationGeneratorLama(model_path)
        if hasattr(generator, 'model'):
            generator.model.eval()
        
        # Process the DataFrame in batches
        for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
            # Create a batch of entries (each is a dict with "doc" and "code")
            batch_entries = df.iloc[i:i+batch_size][["corpus_id", "query_id", "doc", "code"]].to_dict("records")
            
            # Wrap inference in a no_grad context to prevent gradient computations.
            with torch.no_grad():
                batch_explanations = generator.generate_explanations_batch(batch_entries)
            
            for j, explanation_variants in enumerate(batch_explanations):
                for idx, raw_text in enumerate(explanation_variants):
                    # print(raw_text)
                    cleaned_text = clean_output(raw_text,"Answer:")
                    cleaned_text = clean_output(cleaned_text,"</think>")
                    df.loc[i+j, f'explanation_{model_key}_{idx+1}'] = cleaned_text
            
            torch.cuda.empty_cache()
            gc.collect()
        
        del generator
        torch.cuda.empty_cache()
        gc.collect()
    
    # Save the DataFrame with the new explanation columns to a CSV file
    # df.to_csv(output_csv_path, index=False)
    # print(f"\nExplanations from all models have been saved to {output_csv_path}")


Processing models:   0%|          | 0/1 [00:00<?, ?it/s]


Processing model granite...
WARNING 03-26 19:14:20 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-26 19:14:20 config.py:549] This model supports multiple tasks: {'embed', 'generate', 'reward', 'score', 'classify'}. Defaulting to 'generate'.
INFO 03-26 19:14:20 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', speculative_config=None, tokenizer='/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dty

INFO 03-26 19:14:21 model_runner.py:1110] Starting to load model /datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f...



Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.72s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:01<00:00,  1.04it/s]



INFO 03-26 19:14:23 model_runner.py:1115] Loading model weights took 4.7194 GB
INFO 03-26 19:14:24 worker.py:267] Memory profiling takes 0.55 seconds
INFO 03-26 19:14:24 worker.py:267] the current vLLM instance can use total_gpu_memory (44.34GiB) x gpu_memory_utilization (0.90) = 39.91GiB
INFO 03-26 19:14:24 worker.py:267] model weights take 4.72GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.46GiB; the rest of the memory reserved for KV Cache is 34.72GiB.
INFO 03-26 19:14:24 executor_base.py:111] # cuda blocks: 28446, # CPU blocks: 3276
INFO 03-26 19:14:24 executor_base.py:116] Maximum concurrency for 4096 tokens per request: 111.12x
INFO 03-26 19:14:24 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_u

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:21<00:00,  1.60it/s]

INFO 03-26 19:14:46 model_runner.py:1562] Graph capturing finished in 22 secs, took 0.05 GiB
INFO 03-26 19:14:46 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 23.20 seconds




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

















Processed prompts: 100%|██████████| 20/20 [00:08<00:00,  2.43it/s, est. speed input: 1272.40 toks/s, output: 746.10 toks/s]
/tmp/ipykernel_163192/1082677007.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i+j, f'explanation_{model_key}_{idx+1}'] = cleaned_text
/tmp/ipykernel_163192/1082677007.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [89]:
df.head()

,query_id,corpus_id,score,doc,code,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_granite_5
0,q265734,c265608,1,str->list\n Convert XML to URL List.\n From Biligrab.,"def sina_xml_to_url_list(xml_data):\n """"""str->list\n Convert XML to URL List.\n From Biligrab.\n """"""\n rawurl = []\n dom = parseString(xml_data)\n for node in dom.getElementsByTagName('durl'):\n url = node.getElementsByTagName('url')[0]\n rawurl.append(url.childNodes[0].data)\n return rawurl","\nThe provided code snippet is a function named `sina_xml_to_url_list` that converts an XML string into a list of URLs. It uses the `parseString` function to parse the XML data and then iterates through the 'durl' elements in the DOM. For each 'durl' element, it appends the value of the 'url' child node to the `rawurl` list. Finally, it returns the `rawurl` list containing the extracted URLs.",\nThis code snippet is written in Python and it defines a function called `sina_xml_to_url_list` that takes a string `xml_data` as input and returns a list of URLs extracted from the XML data.\n\nThe function first initializes an empty list called `rawurl` to store the extracted URLs. It then creates a DOM (Document Object Model) object by parsing the XML data using the `parseString` function.\n\nThe function then iterates over all the `durl` elements in the DOM using a `for` loop. For each ...,"\nThis code snippet takes XML data as input and converts it into a list of URLs. Here's a simple breakdown:\n\n1. It starts with an empty list called `rawurl`.\n2. It then creates a DOM (Document Object Model) from the XML data.\n3. It goes through each 'durl' node in the XML data.\n4. For each 'durl' node, it finds the 'url' node within it.\n5. It then takes the data from the 'url' node and appends it to the `rawurl` list.\n6. Finally, it returns the `rawurl` list, which contains all the UR...","\nThe code snippet provided is a Python function named `sina_xml_to_url_list` that takes an XML string as input and converts it into a list of URLs. Here's a step-by-step explanation of the code:\n\n1. An empty list named `rawurl` is initialized to store the URLs.\n2. The XML string is parsed using the `parseString` function, which is assumed to be a custom function or library function for parsing XML data. The parsed XML is stored in the `dom` variable.\n3. A loop is initiated to iterate th...","\nThe code snippet in entry['code'] implements the functionality described in the doc string in entry['doc'] by converting XML data to a list of URLs. Here's a step-by-step explanation:\n\n1. The function `sina_xml_to_url_list` takes a single argument, `xml_data`, which is expected to be a string containing XML data.\n2. The function initializes an empty list called `rawurl` to store the extracted URLs.\n3. The XML data is parsed using the `parseString` function, which converts the XML strin..."
1,q265735,c265609,1,Downloads Dailymotion videos by URL.,"def dailymotion_download(url, output_dir='.', merge=True, info_only=False, **kwargs):\n """"""Downloads Dailymotion videos by URL.\n """"""\n\n html = get_content(rebuilt_url(url))\n info = json.loads(match1(html, r'qualities"":({.+?}),""'))\n title = match1(html, r'""video_title""\s*:\s*""([^""]+)""') or \\n match1(html, r'""title""\s*:\s*""([^""]+)""')\n title = unicodize(title)\n\n for quality in ['1080','720','480','380','240','144','auto']:\n try:\n real_...","\nThis Python function, `dailymotion_download`, downloads videos from Dailymotion using a provided URL. It fetches the video's details, extracts the title and available qualities, then downloads the video in the highest quality available. The downloaded video's MIME type, extension, and size are displayed. If the `info_only` parameter is False, the video is also saved to the specified output directory.","\n1. `def dailymotion_download(url, output_dir='.', merge=True, info_only=False, **kwargs):` - This line defines a function named `dailymoti

In [4]:
!export TORCH_USE_CUDA_DSA=1  # Optional: for better error messages during debugging


In [2]:
import pandas as pd
df_codesearchnet=pd.read_csv('explanations/CodeSearchNet_Python_test_vllm.csv')


In [4]:
df_codesearchnet.shape

(14702, 16)

In [6]:
df_codesearchnet.head(5)

,Unnamed: 0,query_id,corpus_id,score,doc,code,explanation_deepseek_1,explanation_deepseek_2,explanation_deepseek_3,explanation_deepseek_4,explanation_deepseek_5,explanation_granite_1,explanation_granite_2,explanation_granite_3,explanation_granite_4,explanation_granite_5
0,0,q265734,c265608,1,str->list\n Convert XML to URL List.\n From Biligrab.,"def sina_xml_to_url_list(xml_data):\n """"""str->list\n Convert XML to URL List.\n From Biligrab.\n """"""\n rawurl = []\n dom = parseString(xml_data)\n for node in dom.getElementsByTagName('durl'):\n url = node.getElementsByTagName('url')[0]\n rawurl.append(url.childNodes[0].data)\n return rawurl",</think>\n\nThe docstring and code snippets represent a function that parses an XML string and extracts URLs from it. The function processes each element of the XML to collect URL data.,"</think>\n\ndef sina_xml_to_url_list(xml_data):\n """"""str->list\n Convert XML to URL List.\n From Biligrab.""""""\n rawurl = []\n dom = parseString(xml_data)\n for node in dom.getElementsByTagName('durl'):\n url = node.getElementsByTagName('url')[0]\n rawurl.append(url.childNodes[0].data)\n return rawurl\n\n1. rawurl = []: Initializes an empty list to store URL data.\n\n2. dom = parseString(xml_data): Parses the input XML string into a DOM node.\n\n3. for node ...","The code takes an XML string as input. It parses this XML into a DOM node. Then, for each element with the tag 'durl', it finds the first child with the tag 'url'. It extracts the data from this URL and appends it to a list. Finally, it returns this list of URLs.\n</think>\n\nThe code takes an XML string, parses it into a DOM node, and for each 'durl' element, extracts the URL data. It returns a list of these URLs.\n\nAnswer: \nThe code takes an XML string, parses it into a DOM node, and for...","Alright, so I've been given this task to figure out how to explain and generate an explanation for a provided code snippet. The code is a function called sina_xml_to_url_list, which takes an XML data as input and converts it into a list of URLs. The explanation needs to not only outline what the code does but also how it can be regenerated based on the provided reasoning. \n\nFirst, I'll start by looking at the code itself. The function initializes an empty list called rawurl. Then, it uses ...","</think>\n\nThe code snippet implements the functionality described in the doc string by parsing the XML data, extracting URL elements, and converting them into a list. It uses the `parseString` method to handle the XML structure, iterates over all `durl` nodes, and collects the corresponding URLs by accessing the `url` elements within each node. The collected URLs are stored in the `rawurl` list, which is returned as the result.\n\nAnswer: \n\nThe code snippet extracts URLs from XML nodes a...","\nThe provided code snippet is a Python function called `sina_xml_to_url_list` that converts XML data to a list of URLs. It uses the `parseString` function to parse the XML data and then iterates through the 'durl' elements in the XML. For each 'durl' element, it appends the value of the 'url' child element to the `rawurl` list. Finally, it returns the `rawurl` list containing the extracted URLs.","\nThis code snippet is written in Python and is used to convert XML data to a list of URLs. Here's a line-by-line explanation of the code:\n\n1. `def sina_xml_to_url_list(xml_data):` - This line defines a function called `sina_xml_to_url_list` that takes one argument, `xml_data`, which is expected to be a string containing XML data.\n2. `""""""str->list` - This is a docstring that describes the purpose of the function. It states that the function converts a string (XML data) to a list of URLs.\...","\nThis code snippet takes an XML string as input and converts it into a list of URLs. It does this by searching for a specific element in the XML called 'durl' and then finding the 'url' element within it. It then extracts the data from the 'url' element and adds it to the list.

In [1]:
import torch
import json
from tqdm import tqdm
import os
import gc
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch
from vllm import LLM, SamplingParams, LLMEngine
from datasets import load_dataset
import csv
from torch.utils.data import Dataset, DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Set environment variable to help with memory fragmentation.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def clean_output(text,keyword):
    # keyword = "Answer:"
    index = text.rfind(keyword)  # Find the last occurrence of "Answer:"
    if index != -1:
        return text[index + len(keyword):].strip()  
    return text

class ExplanationGeneratorLama:
    def __init__(self, model_name, max_new_tokens=500):
        self.max_new_tokens = max_new_tokens
        self.sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=max_new_tokens
        )
        self.llm = LLM(model=model_name, dtype="half")  # GPU half precision

    def generate_explanations_batch(self, entries):
        prompts = []
        for entry in entries:
            code = entry['cleaned_code']
            prompt_templates = [
            
            f"Code snippet: {code}\n"
            "Instruction: Provide a concise explanation of what the above code mean. "
            "Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            , 

            
            f"Code snippet: {code}\n"
            "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            ,

            
            f"Code snippet: {code}\n"
            "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            ,

            
            f"Code snippet: {code}\n"
            "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. "
            "Please give the output just as text only. Do not return anything else.\n"
            "Answer: \n"
            ,

        
            f"Code snippet - entry['code'] : {code}\n" \
            "Instruction: Explain how the code snippet in entry['code'] implements. Please provide the explanation as text only without any additional content.\n" \
            "Answer: \n"
        ]
            prompts.extend(prompt_templates)

        results = self.llm.generate(prompts, self.sampling_params)
        outputs = [res.outputs[0].text for res in results]

        # Group by original entry
        grouped = []
        for i in range(0, len(outputs), 5):
            grouped.append(outputs[i:i+5])

        return grouped


# === CONFIG ===
csv_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/pre_processing_CSN/CodeSearchNet_Python_test_cleaned.csv'
output_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation-Generation/output/CSN_deepseek_test.csv'
checkpoint_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation-Generation/checkpoint/CSN_deepseek_test.txt'
model_path = "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa"
model = "deepseek"
# model_path = "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"
# model = "granite"
batch_size = 128  # adjust to your GPU memory

# === HELPERS ===
def load_checkpoint():
    return int(open(checkpoint_path).read()) if os.path.exists(checkpoint_path) else 0

def save_checkpoint(line_num):
    with open(checkpoint_path, 'w') as f:
        f.write(str(line_num))

def write_header_if_needed(output_path, fieldnames):
    if not os.path.exists(output_path) or os.stat(output_path).st_size == 0:
        with open(output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()

# === MAIN ===
def main():
    dataset = load_dataset('csv', data_files=csv_path, split='train', streaming=True)
    print(f'Dataset loaded - {csv_path}')
    print(f'Output location - {output_path}')

    start_line = load_checkpoint()
    print(f'Starting from line {start_line}')

    generator = ExplanationGeneratorLama(model_path)
    if hasattr(generator, 'model'):
        generator.model.eval()
    print(f'Model loaded - {model_path}')

    buffer = []
    line_num = 0
    written_header = False
    total_len = sum(1 for _ in dataset)

    with open(output_path, 'a', newline='', encoding='utf-8') as outfile:
        writer = None

        for row in tqdm(dataset, desc="Streaming dataset", total=total_len):
            line_num += 1
            if line_num <= start_line:
                continue

            buffer.append(row)

            if len(buffer) == batch_size:
                with torch.no_grad():
                    all_explanations = generator.generate_explanations_batch(buffer)

                for entry, explanations in zip(buffer, all_explanations):
                    for idx, explanation in enumerate(explanations):
                        entry[f'explanation_{model}_{idx+1}'] = explanation

                    # Setup writer on first pass
                    if writer is None:
                        fieldnames = list(entry.keys())
                        write_header_if_needed(output_path, fieldnames)
                        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

                    writer.writerow(entry)

                outfile.flush()
                save_checkpoint(line_num)
                buffer.clear()

        # Final batch (if any)
        if buffer:
            with torch.no_grad():
                all_explanations = generator.generate_explanations_batch(buffer)

            for entry, explanations in zip(buffer, all_explanations):
                for idx, explanation in enumerate(explanations):
                    entry[f'explanation_{model}_{idx+1}'] = explanation

                if writer is None:
                    fieldnames = list(entry.keys())
                    write_header_if_needed(output_path, fieldnames)
                    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

                writer.writerow(entry)

            outfile.flush()
            save_checkpoint(line_num)

    print(f"Finished processing. Output saved to {output_path}")

# === RUN ===
if __name__ == "__main__":
    main()


/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-10 13:03:04,490	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


cuda
Dataset loaded - /work/pi_wenlongzhao_umass_edu/27/anamikaghosh/pre_processing_CSN/CodeSearchNet_Python_test_cleaned.csv
Output location - /work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation-Generation/output/CSN_deepseek_test.csv
Starting from line 0
INFO 04-10 13:03:09 __init__.py:207] Automatically detected platform cuda.
WARNING 04-10 13:03:09 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-10 13:03:28 config.py:549] This model supports multiple tasks: {'score', 'reward', 'embed', 'generate', 'classify'}. Defaulting to 'generate'.
WARNING 04-10 13:03:28 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-10 13:03:28 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=204

[W410 13:03:32.622338224 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.14s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.14s/it]



INFO 04-10 13:03:36 model_runner.py:1115] Loading model weights took 3.3414 GB
INFO 04-10 13:03:38 worker.py:267] Memory profiling takes 1.76 seconds
INFO 04-10 13:03:38 worker.py:267] the current vLLM instance can use total_gpu_memory (47.27GiB) x gpu_memory_utilization (0.90) = 42.54GiB
INFO 04-10 13:03:38 worker.py:267] model weights take 3.34GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 37.75GiB.
INFO 04-10 13:03:38 executor_base.py:111] # cuda blocks: 88364, # CPU blocks: 9362
INFO 04-10 13:03:38 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 10.79x
INFO 04-10 13:03:44 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:20<00:00,  1.71it/s]

INFO 04-10 13:04:04 model_runner.py:1562] Graph capturing finished in 20 secs, took 0.20 GiB
INFO 04-10 13:04:04 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 28.26 seconds


Model loaded - /datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa


Streaming dataset:   0%|          | 1/14702 [00:00<1:09:01,  3.55it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Streaming dataset:   2%|▏         | 255/14702 [01:49<1:43:06,  2.34it/s]


KeyboardInterrupt: 